# SRT Subtitle Generation

This notebook demonstrates how to create SRT subtitles using the unified subtitle module.

In [ ]:
import sys
import pandas as pd
import json
import asyncio
import os
import base64
from pathlib import Path
from typing import List, Dict, Any
from IPython.display import HTML, display

sys.path.append("../")

# Import the necessary modules from the new subtitles package
from src.subtitles import create_track

## Creating SRT Subtitles

Let's use the new unified subtitle module to create SRT subtitles from a transcript file.

In [ ]:
# Path to the transcript file
transcript_file = Path(
    "../data/transcripts/regular_council_meeting___2025_02_26.diarized.json"
)
# Create an SRT track from the transcript
srt_track = create_track(
    transcript_data=transcript_file,
    format='srt',
    max_duration=5.0,  # Maximum duration for each subtitle
    max_length=80,  # Maximum length in characters
    include_speaker_prefix=True,  # Include speaker labels
)

# Get the SRT content
srt_content = srt_track.content()

# Display the first few entries
print("\n".join(srt_content.split("\n\n")[:5]))

## Saving the SRT File

Let's save the generated SRT content to a file.

In [ ]:
# Define output directory
output_dir = Path("../data")
output_dir.mkdir(exist_ok=True)

# Output SRT file path
srt_file_path = output_dir / "subtitles.srt"

# Save the SRT content to a file
with open(srt_file_path, "w", encoding="utf-8") as f:
    f.write(srt_content)

print(f"SRT file saved to: {srt_file_path}")

In [ ]:
# Create video directory
video_dir = Path("../data/video")
video_dir.mkdir(exist_ok=True)

# Check if we need to download the video
input_video_path = video_dir / "regular_council_meeting___2025_02_26.mp4"

# Display file info
file_size_mb = input_video_path.stat().st_size / (1024 * 1024)
print(f"File size: {file_size_mb:.2f} MB")

In [ ]:
# Import the subtitles module
from src.subtitles import create_track
import subprocess

# Path to the SRT file we saved earlier
srt_file = Path("../data/subtitles.srt")

# Path for the output video with embedded subtitles
output_video = video_dir / "regular_council_meeting___2025_02_26_with_srt_subtitles.mp4"

# Check if FFmpeg is installed

cmd = [
    "ffmpeg",
    "-y",  # Overwrite output files without asking
    "-i",
    str(input_video_path),  # Input video
    "-vf",
    f"subtitles={str(srt_file)}:force_style='FontSize=24,Alignment=2'",  # Add subtitles
    "-c:v",
    "libx264",  # Video codec
    "-crf",
    "23",  # Quality (lower is better)
    "-c:a",
    "copy",  # Copy audio stream
    "-t",
    "300",  # Limit to first five minutes for testing
    str(output_video),  # Output path
]

print("Running FFmpeg command to embed subtitles...")
print(" ".join(cmd))
result = subprocess.run(
    cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE
)
print("Successfully created video with embedded subtitles!")
print(f"Output video saved to: {output_video}")



In [ ]:
# Display the video with embedded subtitles using IPython's Video widget
from IPython.display import Video

# Path to the video with embedded subtitles
if output_video.exists():
    # Display the video
    print("Playing video with embedded subtitles:")
    display(Video(str(output_video), embed=True, width=640, height=360))
else:
    print(f"Video file not found: {output_video}")